In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# initialize the model
model = Sequential()

# add first convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 3)))

# add second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))

# add max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# add third convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))

# add fourth convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))

# add fifth convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))

# add flatten layer
model.add(Flatten())

# add first dense layer
model.add(Dense(1024, activation='relu'))

# add dropout layer for regularization
model.add(Dropout(0.5))

# add second dense layer
model.add(Dense(10, activation='softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
import pandas as pd

# Assuming you have a CSV file with a header, e.g., ['feature1', 'feature2', ..., 'target1']

# Read the CSV file into a pandas DataFrame
data = pd.read_csv('path_to_your_csv_file.csv')

# Separate the features and targets
features = data.iloc[:, :-1]
targets = data.iloc[:, -1]

# Create the training and testing sets
# Here, we are assuming you have 80% of your data for training and 20% for testing
train_features = features.iloc[:int(0.8*len(features))]
test_features = features.iloc[int(0.8*len(features)):]

train_targets = targets.iloc[:int(0.8*len(targets))]
test_targets = targets.iloc[int(0.8*len(targets)):]

# Reshape your target variable to match the dimensions of your input layer
train_targets = train_targets.values.reshape(-1, 641, 1)
test_targets = test_targets.values.reshape(-1, 641, 1)

# Now, you can use these training and testing sets to train your model

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

# Load EEG data from CSV
eeg_data = pd.read_csv(r"C:\Users\HP\Desktop\Dataset\Model_Input_2sec\LR_2sec.csv")

# Define the number of rows in each epoch (4 seconds)
epoch_length = 321

# Initialize lists to store epochs and labels
epochs = []
labels = []

# Extract EEG data and labels
for i in range(0, len(eeg_data), epoch_length):
    epoch = eeg_data.iloc[i:i + epoch_length]
    if len(epoch) == epoch_length:
        eeg_epoch = epoch.iloc[:, 1:-1].values  # Assuming EEG channels are from column 1 to the second-to-last column
        label = epoch.iloc[0, -1]  # Assuming the target is in the last column
        epochs.append(eeg_epoch)
        labels.append(label)

# Convert to NumPy arrays
epochs = np.array(epochs)
labels = np.array(labels)

# Perform label encoding for multi-class classification
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(epochs, labels_encoded, test_size=0.3, random_state=42)

# Define EEGNet architecture
model = keras.Sequential()
model.add(layers.Conv2D(8, (1, 64), input_shape=(epochs.shape[1], epochs.shape[2], 1), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv2D(kernel_size=(epochs.shape[1], 1), depth_multiplier=2, depthwise_initializer='he_normal', use_bias=False, padding='valid'))
model.add(layers.BatchNormalization())
model.add(layers.DepthwiseConv2D(kernel_size=(1, 16), depth_multiplier=2, depthwise_initializer='he_normal', use_bias=False, padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('elu'))
model.add(layers.AveragePooling2D(pool_size=(1, 4)))
model.add(layers.Dropout(0.5))
model.add(layers.SeparableConv2D(16, (1, 16), activation='elu', depthwise_initializer='he_normal', use_bias=False, padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.SeparableConv2D(16, (1, 16), activation='elu', depthwise_initializer='he_normal', use_bias=False, padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.AveragePooling2D(pool_size=(1, 4)))  # Adjust the pool_size as needed
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(3, activation='softmax'))  # 3 classes (Left, Right, Rest)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape data to match the input shape
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

# Train the model
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


ValueError: Exception encountered when calling layer "depthwise_conv2d" (type DepthwiseConv2D).

Negative dimension size caused by subtracting 1 from 0 for '{{node depthwise_conv2d/depthwise}} = DepthwiseConv2dNative[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1]](Placeholder, depthwise_conv2d/depthwise/ReadVariableOp)' with input shapes: [?,4885,0,8], [4885,1,8,2].

Call arguments received by layer "depthwise_conv2d" (type DepthwiseConv2D):
  • inputs=tf.Tensor(shape=(None, 4885, 0, 8), dtype=float32)